In [28]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

In [29]:
df = pd.read_excel('get_around_delay_analysis.xlsx')


### Quelques points de clarification sur les colonnes du dataset :

rental_id : identifiant unique de la location.

car_id : identifiant unique de la voiture.

checkin_type : mode d’enregistrement et de départ (accès et restitution du véhicule) :

- mobile : contrat de location signé sur le smartphone du propriétaire.

- connect : voiture équipée de la technologie Connect, ouverte directement par le conducteur avec son smartphone.

state : statut de la commande (terminée ou annulée).

delay_at_checkout_in_minutes : différence (en minutes) entre l’heure de fin prévue et l’heure réelle de restitution.

- Une valeur négative = restitution en avance.

- Une valeur positive = retard.

previous_ended_rental_id : identifiant de la location précédente (NULL si absence de location précédente ou si le délai > 12 heures).

time_delta_with_previous_rental_in_minutes : temps (en minutes) entre la fin prévue de la location précédente et le début prévu de la location courante.

Valeur uniquement renseignée si < 12 heures, sinon NULL.

In [30]:
display(df.describe(include='all'), df.info(), df.isna().sum(), df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


None

rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


De nombreux manquants sur previous_ended_rental_id et time_delta_with_previous_rental_in_minutes sont dus au fait que le véhicule n’ait pas encore été reloué.

delay_at_checkout_in_minutes présente une forte disparitée et des valeurs aberrantes (ex. 71 084 min ≈ 49 jours, -22 433 min ≈ -15,6 jours) à filtrer.

25 % des retours sont en avance de 36 min. La moitier des retour ont un maximum de 9 min de retard, et cela monte jusqu'a 67 min pour 75% des retour. 

Pour time_delta_with_previous_rental_in_minutes, le temps moyen entre 2 locations est de 279 min (4h40). 25% des véhicules sont reloué en moins de 60 min et cela monte jusqu'a 720 min (12h)


In [31]:
fig = px.pie(df, names="state", title="Répartition par état")
fig.show()

In [32]:
fig = px.histogram(df, x='checkin_type', color='state', barmode="group", title="Distribution des status par méthode de commande", labels={"checkin_type":"Methode"})
fig.show()


Environ 15 % des transactions sont annulées. La proportion d’annulations est légèrement plus élevée avec l’utilisation de Connect qu’avec le check-in mobile.
Une hypothèse est que l’absence d’échange humain dans la procédure Connect pourrait rendre l’annulation plus facile pour certains utilisateurs, mais cela mériterait une analyse complémentaire. Observons la distribution des méthodes de locations.

In [33]:
fig = px.histogram(
    df.query("delay_at_checkout_in_minutes >= -150 and delay_at_checkout_in_minutes <= 150"),
    x="delay_at_checkout_in_minutes",
    nbins=60,
    title="Distribution des retards (fenêtre -150 à 150 min)"
)

fig.show()


In [34]:
fig = px.histogram(
    df.query("-150 <= delay_at_checkout_in_minutes <= 150"),
    x="delay_at_checkout_in_minutes",
    color="checkin_type",
    nbins=int((150 - (-150)) / 10),  # ≈ 13 bacs ~30 min
    opacity=0.2,
    title="Distribution des retards par type de check-in",
    labels={"delay_at_checkout_in_minutes": "Retard (minutes)"},
    range_x=[-150, 150],
    width=1200, height=600,
)
fig.update_layout(barmode="overlay")
fig.update_yaxes(title="Nombre de locations")
# fig.add_vline(x=0, line_color="red", line_dash="dash", opacity=0.5)
fig.show()

20% des locations sont réalisés via l'application mobile historique. Les 2 types de locations subissent des retards, cependant on peut observer un pic de fréquence entre 0 et 50 min pour le mobile alors que le connect reste stable. Observons la distribution de chacune de ces methodes de location.

In [35]:
df["delay_clipped"] = df["delay_at_checkout_in_minutes"].clip(-150, 150)

fig = px.box(
    df.query("-150 <= delay_at_checkout_in_minutes <= 150"),
    x="checkin_type",
    y="delay_clipped",
    points="all",
    labels={
        "checkin_type": "Méthode",
        "delay_clipped": "Retard (minutes, borné à [-150, 150])"
    },
    title="Distribution des retards par méthode de check-in (clippé)"
)
fig.show()
df.drop("delay_clipped", axis=1, inplace=True)

Les locations via Connect sont plus ponctuelles que les locations mobile. Elles sont même plus souvent en avance. Observons maintenant le delai entre location.

In [36]:
fig = px.histogram(
    df,
    x="time_delta_with_previous_rental_in_minutes",
    color="checkin_type",
    nbins=20,  
    labels={
        "time_delta_with_previous_rental_in_minutes": "Delai en min",
        "checkin_type": "Méthode"
    },
    title="Distribution du delai entre location"
)

fig.update_layout(barmode="group")      # superposer les histogrammes
fig.update_yaxes(title="Nombre de locations")
fig.show()

Le délai entre deux locations est globalement court : la distribution présente un pic autour de 0–40 min, puis décroît jusqu’à 300 min. Elle reste ensuite faible et plate, puis remonte légèrement au-delà de 600 min.
À ce stade, la méthode de check-in ne semble pas liée au délai entre locations : les distributions par méthode sont très proches.

## Premier constat:
Les données présentent des valeurs manquantes logiques.

La majorité des retards est courte : 50 % ≤ 9 min, 75 % ≤ 67 min, mais certains cas très longs existent.

Le délai moyen entre deux locations est de ~4h40, avec 25 % reloués en < 1h, ce qui accentue le risque de conflits.

15 % des transactions sont annulées, un peu plus via Connect que via mobile.

Connect apparaît plus ponctuel et plus souvent en avance que le mobile.

Le délai entre deux locations ne varie pas selon la méthode de check-in.

Prochaine étape : analyse des retards et conséquence

### Points sur les annulations

In [37]:
print("Le nombre d'annulations est de", (df["state"] == "canceled").sum())

Le nombre d'annulations est de 3265


In [38]:
mask = df['previous_ended_rental_id'] > 0
df_nan = df[mask]
print("Le nombre d'annulations est de", (df_nan["state"] == "canceled").sum())

Le nombre d'annulations est de 229


Nous avons identifié 3 265 annulations au total. Parmi celles-ci, 229 correspondent à des locations reliées à une précédente réservation (elles possèdent un previous_ended_rental_id).
Ces cas représentent donc des annulations qui s’enchaînent directement derrière une autre location et peuvent être impactées par un retard de restitution.
Pour les autres annulations, sans lien avec un précédent véhicule, elles ne sont pas liées à un retard entre deux locations.
Nous allons donc concentrer notre analyse sur ces 229 annulations.

In [39]:
# On crée un dataframe locations annulées
canceled = df[df["state"] == "canceled"][["rental_id", "car_id", "previous_ended_rental_id", "checkin_type","time_delta_with_previous_rental_in_minutes"]]

# On rattache la location annulée à la précédente
merged = canceled.merge(
    df[["rental_id", "car_id", "delay_at_checkout_in_minutes", "checkin_type"]],
    left_on="previous_ended_rental_id",   # clé : location précédente
    right_on="rental_id",                 
    suffixes=("_canceled", "_previous")
)
# Regardons les conflits direct c'est à dire que la restitution est supérieur au delay entre location prévu
conflict_strict = merged[
    merged["delay_at_checkout_in_minutes"] > merged["time_delta_with_previous_rental_in_minutes"]
]
print("Conflits stricts :", len(conflict_strict))
print("Moyenne retard précédent (conflits seulement) :", conflict_strict["delay_at_checkout_in_minutes"].mean().round(2))

Conflits stricts : 37
Moyenne retard précédent (conflits seulement) : 253.76


Sur les 229 annulations, 37 sont directement liées à un retard de restitution dépassant l’heure prévue de la prochaine location.
La moyenne du retard dans ces cas est de 253 minutes (~4h15), ce qui constitue un retard très important. Il est logique que le client suivant ait annulé sa réservation dans de telles conditions.
Ce type de problème relève davantage d’une gestion client (retards extrêmes) et nécessitera des actions spécifiques pour le limiter.
Dans notre objectif, nous allons plutôt nous concentrer sur les annulations liées à des restitutions proches de l’heure de la prochaine location, afin d’analyser le comportement des clients et identifier des leviers pour réduire ce phénomène.

In [40]:
# Cas "conflit -30min"
conflict_minus30 = merged[
    merged["delay_at_checkout_in_minutes"] > merged["time_delta_with_previous_rental_in_minutes"] - 30
]

# Cas "conflit -60min"
conflict_minus60 = merged[
    merged["delay_at_checkout_in_minutes"] > merged["time_delta_with_previous_rental_in_minutes"] - 60
]

conflict_minus90 = merged[
    merged["delay_at_checkout_in_minutes"] > merged["time_delta_with_previous_rental_in_minutes"] - 90
]

conflict_minus180 = merged[
    merged["delay_at_checkout_in_minutes"] > merged["time_delta_with_previous_rental_in_minutes"] - 180
]
print("Conflits -30min :", len(conflict_minus30))
print("Conflits -60min :", len(conflict_minus60))
print("Conflits -90min :", len(conflict_minus90))
print("Conflits -180min :", len(conflict_minus180))

# Moyenne des retards dans chaque cas
print("\nMoyenne des retards :")

print("-30min :", conflict_minus30["delay_at_checkout_in_minutes"].mean())
print("-60min :", conflict_minus60["delay_at_checkout_in_minutes"].mean())
print("-90min :", conflict_minus90["delay_at_checkout_in_minutes"].mean())
print("-180min :", conflict_minus180["delay_at_checkout_in_minutes"].mean())


Conflits -30min : 47
Conflits -60min : 50
Conflits -90min : 57
Conflits -180min : 89

Moyenne des retards :
-30min : 204.2127659574468
-60min : 197.76
-90min : 171.9298245614035
-180min : 114.46067415730337


Si on regarde 30 miniutes avant le début de la location du véhicule combien ont été annulé (buffer de 30 minutes) : ces cas correspondent à des locations rendues trop proches de la suivante (<30 min de marge).

Avec un buffer de 60 minutes, seulement 3 cas en plus → ce sont donc des situations rares mais critiques.

En élargissant à 90 minutes, on ajoute encore 7 cas.

Enfin, avec 3h de buffer, on attrape beaucoup plus de cas (32 supplémentaires), mais ce sont des retards plus modérés, pas toujours absorbables sans perte de revenu.

Observons maintenant quelle méthode de location est la plus impactée par ces annulations.

In [41]:
# Dans le cas de l'application mobile
mask = merged["checkin_type_canceled"] == "mobile"
merged_mob = merged[mask]
print("Nombre d'annulation sur mobile", (merged_mob["delay_at_checkout_in_minutes"].size))
conflict_strict1= merged_mob[
    merged_mob["delay_at_checkout_in_minutes"] > merged_mob["time_delta_with_previous_rental_in_minutes"]
]
print("Conflits stricts :", len(conflict_strict1))
print("Moyenne retard précédent (conflits seulement) :", conflict_strict1["delay_at_checkout_in_minutes"].mean().round(2))

# Cas "conflit -30min"
conflict_1minus30 = merged_mob[
    merged_mob["delay_at_checkout_in_minutes"] > merged_mob["time_delta_with_previous_rental_in_minutes"] - 30
]

# Cas "conflit -60min"
conflict_1minus60 = merged_mob[
    merged_mob["delay_at_checkout_in_minutes"] > merged_mob["time_delta_with_previous_rental_in_minutes"] - 60
]

conflict_1minus90 = merged_mob[
    merged_mob["delay_at_checkout_in_minutes"] > merged_mob["time_delta_with_previous_rental_in_minutes"] - 90
]

conflict_1minus180 = merged_mob[
    merged_mob["delay_at_checkout_in_minutes"] > merged_mob["time_delta_with_previous_rental_in_minutes"] - 180
]
print("Résulat avec l'application mobile")
print("Conflits -30min :", len(conflict_1minus30))
print("Conflits -60min :", len(conflict_1minus60))
print("Conflits -90min :", len(conflict_1minus90))
print("Conflits -180min :", len(conflict_1minus180))

# Moyenne des retards dans chaque cas
print("\nMoyenne des retards :")

print("-30min :", conflict_1minus30["delay_at_checkout_in_minutes"].mean())
print("-60min :", conflict_1minus60["delay_at_checkout_in_minutes"].mean())
print("-90min :", conflict_1minus90["delay_at_checkout_in_minutes"].mean())
print("-180min :", conflict_1minus180["delay_at_checkout_in_minutes"].mean())


Nombre d'annulation sur mobile 98
Conflits stricts : 18
Moyenne retard précédent (conflits seulement) : 253.78
Résulat avec l'application mobile
Conflits -30min : 20
Conflits -60min : 21
Conflits -90min : 26
Conflits -180min : 42

Moyenne des retards :
-30min : 228.25
-60min : 218.0952380952381
-90min : 174.30769230769232
-180min : 121.61904761904762


In [42]:
# Dans le cas de l'utilisation de connect
mask = merged["checkin_type_canceled"] == "connect"
merged_connect = merged[mask]
print("Nombre d'annulation sur connect", (merged_connect["delay_at_checkout_in_minutes"].size))
conflict_strict2 = merged_connect[
    merged_connect["delay_at_checkout_in_minutes"] > merged_connect["time_delta_with_previous_rental_in_minutes"]
]
print("Conflits stricts :", len(conflict_strict2))
print("Moyenne retard précédent (conflits seulement) :", conflict_strict2["delay_at_checkout_in_minutes"].mean().round(2))

# Cas "conflit -30min"
conflict_2minus30 = merged_connect[
    merged_connect["delay_at_checkout_in_minutes"] > merged_connect["time_delta_with_previous_rental_in_minutes"] - 30
]

# Cas "conflit -60min"
conflict_2minus60 = merged_connect[
    merged_connect["delay_at_checkout_in_minutes"] > merged_connect["time_delta_with_previous_rental_in_minutes"] - 60
]

conflict_2minus90 = merged_connect[
    merged_connect["delay_at_checkout_in_minutes"] > merged_connect["time_delta_with_previous_rental_in_minutes"] - 90
]

conflict_2minus180 = merged_connect[
    merged_connect["delay_at_checkout_in_minutes"] > merged_connect["time_delta_with_previous_rental_in_minutes"] - 180
]
print("Résulat avec l'utilisation de connect")
print("Conflits -30min :", len(conflict_2minus30))
print("Conflits -60min :", len(conflict_2minus60))
print("Conflits -90min :", len(conflict_2minus90))
print("Conflits -180min :", len(conflict_2minus180))

# Moyenne des retards dans chaque cas
print("\nMoyenne des retards :")

print("-30min :", conflict_2minus30["delay_at_checkout_in_minutes"].mean())
print("-60min :", conflict_2minus60["delay_at_checkout_in_minutes"].mean())
print("-90min :", conflict_2minus90["delay_at_checkout_in_minutes"].mean())
print("-180min :", conflict_2minus180["delay_at_checkout_in_minutes"].mean())


Nombre d'annulation sur connect 131
Conflits stricts : 19
Moyenne retard précédent (conflits seulement) : 253.74
Résulat avec l'utilisation de connect
Conflits -30min : 27
Conflits -60min : 29
Conflits -90min : 31
Conflits -180min : 47

Moyenne des retards :
-30min : 186.40740740740742
-60min : 183.0344827586207
-90min : 169.93548387096774
-180min : 108.06382978723404


Les deux systèmes subissent des annulations liées aux délais courts.

Connect est plus fréquent en volume absolu, mais ses retards sont légèrement moins sévères.

Mobile présente moins de cas mais avec des retards plus longs → donc plus difficiles à absorber sans impact client.

Les deux méthodes sont exposées aux retards, mais Connect encaisse plus de cas tandis que Mobile concentre les retards les plus importants.

## Pour conclure
La majorité des annulations ne sont pas dues aux retards, mais parmi celles qui le sont, elles surviennent principalement lors de retards extrêmes ou de délais très courts entre deux locations. Un buffer opérationnel de 30 à 60 minutes pourrait réduire les annulations critiques, sans trop impacter la disponibilité des véhicules. 
Avant d’évaluer l’impact business, faisons un point sur le fichier pricing afin de conclure par une analyse chiffrée de cet impact, en lien avec les éléments que nous venons de mettre en évidence.

## Analyse du pricing

In [43]:
df_pricing = pd.read_csv('get_around_pricing_project.csv')

In [44]:
display(df_pricing.describe(include='all'), df_pricing.info(), df_pricing.head(10), df_pricing.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 4843 non-null   int64 
 1   model_key                  4843 non-null   object
 2   mileage                    4843 non-null   int64 
 3   engine_power               4843 non-null   int64 
 4   fuel                       4843 non-null   object
 5   paint_color                4843 non-null   object
 6   car_type                   4843 non-null   object
 7   private_parking_available  4843 non-null   bool  
 8   has_gps                    4843 non-null   bool  
 9   has_air_conditioning       4843 non-null   bool  
 10  automatic_car              4843 non-null   bool  
 11  has_getaround_connect      4843 non-null   bool  
 12  has_speed_regulator        4843 non-null   bool  
 13  winter_tires               4843 non-null   bool  
 14  rental_p

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


None

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183
5,5,Citroën,152352,225,petrol,black,convertible,True,True,False,False,True,True,True,131
6,6,Citroën,205219,145,diesel,grey,convertible,True,True,False,False,True,True,True,111
7,7,Citroën,115560,105,petrol,white,convertible,True,True,False,False,False,True,True,78
8,8,Peugeot,123886,125,petrol,black,convertible,True,False,False,False,False,True,True,79
9,9,Citroën,139541,135,diesel,white,convertible,False,False,False,False,True,False,True,132


Unnamed: 0                   0
model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64


Le fichier contient 4 843 véhicules sans valeurs manquantes. Cependant, certaines anomalies apparaissent : un mileage négatif et des kilométrages extrêmement élevés (>1 000 000 km), ainsi que des prix de location par jour très bas (min = 10$) comparés à une moyenne de 121$ et pour finir une puissance moteur à 0. Ces valeurs devront être vérifiées et potentiellement exclues pour éviter de biaiser les analyses.




In [45]:
# Regardons les 5 valeurs les plus importantes de la colonne
df_pricing["mileage"].nlargest(5)

3732    1000376
557      484615
2350     477571
2829     439060
3198     405816
Name: mileage, dtype: int64

In [46]:
# Suppression des valeurs mileage extreme et engine power à 0
df_pricing = df_pricing[df_pricing['mileage'] >=0]
df_pricing = df_pricing[df_pricing['mileage'] <= 1000000]
df_pricing = df_pricing[df_pricing['engine_power'] > 0 ]

In [47]:
# Suppression de la colonne Unnamed: 0
df_pricing.drop('Unnamed: 0', axis=1, inplace=True)

In [48]:
df_pricing.shape

(4840, 14)

In [49]:
df_pricing.to_csv('pricing_clean.csv', index=False)

In [50]:
fig = px.histogram(df_pricing, x="rental_price_per_day", nbins=50,
                   title="Distribution des prix de location par jour")
fig.show()

In [51]:
fig = px.box(df_pricing, x="car_type", y="rental_price_per_day",
             title="Prix par type de véhicule")
fig.show()



In [52]:
fig = px.box(df_pricing, x="fuel", y="rental_price_per_day",
             title="Prix selon le type de carburant")
fig.show()


le marché est assez concentré autour de 100–140 €/jour.
Les SUV, coupés et cabriolets ont en moyenne des prix plus élevés.

Les citadines (subcompact) et hatchbacks sont en bas de l’échelle (~100 €/jour).

Les voitures hybrides se louent plus cher en moyenne.

Les diesel et essence forment le gros du marché, avec des prix plus dispersés.

Les véhicules électriques sont peu nombreux dans l’échantillon mais présentent des prix homogènes (~120–130 €/jour).

In [ ]:
df_corr = df_pricing.copy()
bool_cols = df_corr.select_dtypes(include="bool").columns
df_corr[bool_cols] = df_corr[bool_cols].astype(int)

num_df = df_corr.select_dtypes(include=["int64", "float64"])

corr = num_df.corr()

fig = px.imshow(
    corr,
    text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matrice de corrélation",
    height=800,
    width=1000
)
fig.show()


La matrice de corrélation met en évidence :

une forte corrélation positive entre le prix de location et la puissance du moteur,

une corrélation négative entre le kilométrage et le prix, ce qui confirme que les véhicules plus anciens ou plus utilisés se louent moins cher.

On observe également que certains équipements influencent le prix : la présence d’un GPS et le type de transmission automatique sont associés à des tarifs plus élevés.

In [54]:
brand_counts = df_pricing["model_key"].value_counts().reset_index()
brand_counts.columns = ["model_key", "count"]

fig = px.bar(
    brand_counts,
    x="model_key",
    y="count",
    title="Nombre de véhicules par marque",
    labels={"model_key": "Marque", "count": "Nombre de véhicules"}
)
fig.show()


In [55]:
avg_engine = df_pricing.groupby("model_key")['engine_power'].mean().reset_index()
fig = px.bar(
    avg_engine.sort_values("engine_power", ascending=False),
    x="model_key",
    y="engine_power",
    title="Puissance moyenne par marque de véhicule",
    labels={"model_key": "Marque", "engine_power": "Pissance en CV"}
)
fig.show()

In [56]:
avg_price = df_pricing.groupby("model_key")["rental_price_per_day"].mean().reset_index()


fig = px.bar(
    avg_price.sort_values("rental_price_per_day", ascending=False),
    x="model_key",
    y="rental_price_per_day",
    title="Prix moyen par marque",
    labels={"model_key": "Marque", "rental_price_per_day": "Prix moyen (€)"}
)
fig.show()


Nous constatons que cinq marques dominent le parc : Citroën, Renault, BMW, Peugeot et Audi.
Le prix moyen pratiqué par Citroën fait partie des plus bas, avec 108,7$, soit environ 10$ en dessous du prix moyen global. Peugeot est encore moins cher, avec une moyenne de 105$.

Renault et BMW se situent autour de la moyenne, avec respectivement 120$ et 117$, tandis qu’Audi est au-dessus avec 130,5$.

Enfin, Suzuki, Mini et Lexus affichent les tarifs moyens les plus élevés (jusqu’à 223 $), mais ces marques sont peu représentées. Elles font néanmoins partie du Top 3 des véhicules ayant les plus grosses cylindrées.

### Pour conclure
L’analyse des prix de location montre que le marché est globalement concentré autour de 100 à 140 $/jour, avec des variations selon le type de véhicule, la motorisation et la marque. Certains segments (SUV, coupés, hybrides) se louent plus cher, tandis que les citadines et hatchbacks restent en bas de l’échelle. Nous avons également constaté une corrélation nette : les véhicules puissants se louent plus cher, alors qu’un kilométrage élevé réduit le prix.

Ces éléments sont essentiels pour estimer l’impact financier des annulations. En effet, chaque annulation représente une perte potentielle de revenu journalier, proportionnelle au prix moyen du véhicule concerné.

Dans la suite de notre analyse, nous allons donc mesurer la perte économique liée aux annulations et simuler l’effet d’une politique opérationnelle consistant à augmenter le buffer entre deux locations. L’objectif sera de trouver un compromis entre :

Réduire les annulations (et donc sécuriser le revenu),

Sans trop réduire la disponibilité des véhicules sur la plateforme.

### Analyse Business
Nous avons constaté que le prix moyen de location est de 121 $ et que 229 locations ont été annulées en raison d’un retard.
En combinant ces éléments avec notre analyse du buffer, nous allons maintenant évaluer comment réduire cette perte potentielle tout en préservant un niveau de service client optimal.


In [ ]:
annulation = (df_nan["state"] == "canceled").sum() 
loc_moyenne = df_pricing['rental_price_per_day'].mean()
print("La perte du aux annulations s'élève en moyenne a :", (annulation * loc_moyenne).round(2),"$")

La perte du aux annulations s'élève en moyenne a : 27755.51 $


In [58]:

print("Pour 37 d'entre elle c'est du a un retard qui dépasse l'heure prévu de la prochaine location, ce qui représente :", (loc_moyenne * len(conflict_strict)).round(2),"$")
print("Pour un delai proche de l'heure de la prochaine location inférieur à 30 min, cela représente  :", (loc_moyenne * (len(conflict_minus30)-len(conflict_strict))).round(2))
print("Pour un delai proche de l'heure de la prochaine location inférieur à 60 min, cela représente  :", (loc_moyenne * (len(conflict_minus60)-len(conflict_strict))).round(2))
print("Pour un delai proche de l'heure de la prochaine location inférieur à 90 min, cela représente  :", (loc_moyenne * (len(conflict_minus90)-len(conflict_strict))).round(2))
print("Pour un delai proche de l'heure de la prochaine location inférieur à 180 min, cela représente :", (loc_moyenne * (len(conflict_minus180)-len(conflict_strict))).round(2))

Pour 37 d'entre elle c'est du a un retard qui dépasse l'heure prévu de la prochaine location, ce qui représente : 4484.51 $
Pour un delai proche de l'heure de la prochaine location inférieur à 30 min, cela représente  : 1212.03
Pour un delai proche de l'heure de la prochaine location inférieur à 60 min, cela représente  : 1575.64
Pour un delai proche de l'heure de la prochaine location inférieur à 90 min, cela représente  : 2424.06
Pour un delai proche de l'heure de la prochaine location inférieur à 180 min, cela représente : 6302.56


La mise en place d’un buffer de 30 minutes permettrait d’éviter environ 1 200$ de pertes liées aux annulations. Avec un buffer de 60 minutes, le gain supplémentaire reste limité (environ 300$).

Les gains deviennent plus significatifs à partir de 180 minutes (≈ 6 300$), mais un tel délai serait difficilement acceptable pour le business, car il réduirait fortement la disponibilité des véhicules.

Observons maintenant l’impact concret de l’ajout d’un buffer de 30 minutes entre la restitution du véhicule et sa prochaine location, à partir des données dont nous disposons.

In [59]:
# On filtre uniquement les locations terminées
df_valid = df[df["state"] == "ended"].copy()

# On Calcule l'écart
df_valid["ecart"] = df_valid["time_delta_with_previous_rental_in_minutes"] - df_valid["delay_at_checkout_in_minutes"]

# Fonction pour compter les cas bloqués
def count_blocked(df, buffer):
    return (df["ecart"] < buffer).sum()

# Simulation pour différents buffers
CA_total = (df['state'] == 'ended').sum() * loc_moyenne

for b in [30, 60, 90, 180]:
    blocked = count_blocked(df_valid, b)
    perte_estimee = blocked * loc_moyenne
    print(f"Buffer {b} min -> {blocked} locations bloquées -> perte estimée : {perte_estimee:.2f} $ -> {perte_estimee / CA_total:.2%} du revenu total")


Buffer 30 min -> 340 locations bloquées -> perte estimée : 41209.05 $ -> 1.88% du revenu total
Buffer 60 min -> 426 locations bloquées -> perte estimée : 51632.52 $ -> 2.36% du revenu total
Buffer 90 min -> 516 locations bloquées -> perte estimée : 62540.80 $ -> 2.86% du revenu total
Buffer 180 min -> 736 locations bloquées -> perte estimée : 89205.48 $ -> 4.08% du revenu total


Dans le cas de l’utilisation d’un buffer de 30 minutes, nous prenons le risque de bloquer environ 340 locations ayant justement un écart de 30 minutes entre deux réservations.

Les pertes potentielles sont alors nettement supérieures aux gains apportés par cette solution.

Pour rappel, la perte totale liée aux annulations dues aux retards est estimée à 27 751$, tandis que le gain potentiel lié à l’ajout d’un buffer de 30 minutes ne serait que de 1 211$.

Il n’est donc pas économiquement viable de mettre en place un buffer de 30 minutes de manière généralisée.

## Pour conclure

Nos analyses montrent que les annulations liées aux retards coûtent cher à l’entreprise, mais elles restent relativement limitées (229 cas identifiés).
L’ajout d’un buffer de 30 minutes permettrait de réduire ces annulations, mais les gains sont très inférieurs aux pertes générées par la baisse de disponibilité des véhicules.
Il n’est donc pas recommandé d’instaurer un buffer global.

En revanche, plusieurs alternatives peuvent être envisagées pour réduire ce phénomène sans effets négatifs majeurs :

Buffer ciblé (à étudier) :

Lorsque le délai prévu entre deux locations est très court (ex. < 1h).

Pour certains profils de véhicules plus exposés (forte demande, zones tendues).

Mesures opérationnelles :

Notifications automatiques pour inciter à rendre le véhicule en avance.

Pénalités financières pour retards répétés (option plus extrême et potentiellement anticommerciale).

Enfin, de nouvelles données permettraient d’affiner l’analyse et les recommandations, en particulier :

Date et heure de début/fin des locations.

Durée réelle des locations.